# Cleaning the data ...

Libraries for Project ...

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from meteostat import Point, Hourly

Load Data ...

In [2]:
Org_data = pd.read_csv("/home/malaka/Projects/CV_Projects/Datasets/Org_data.csv")
Org_data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,AA,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,IC,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,IC,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,IC,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387
4,200412582,09/09/2020 12:00:00 AM,09/09/2020 12:00:00 AM,630,4,Hollenbeck,413,1,510,VEHICLE - STOLEN,...,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E AVENUE 28,NaN,34.0820,-118.2130


Handle Missing Values

In [3]:
print(Org_data.isnull().sum())

# Example: Drop rows with missing latitude or longitude (important for geospatial analysis)
df = Org_data.dropna(subset=['LAT', 'LON'])

DR_NO                  0
Date Rptd              0
DATE OCC               0
TIME OCC               0
AREA                   0
AREA NAME              0
Rpt Dist No            0
Part 1-2               0
Crm Cd                 0
Crm Cd Desc            0
Mocodes           147473
Vict Age               0
Vict Sex          140607
Vict Descent      140619
Premis Cd             16
Premis Desc          582
Weapon Used Cd    661838
Weapon Desc       661838
Status                 1
Status Desc            0
Crm Cd 1              11
Crm Cd 2          918239
Crm Cd 3          984577
Crm Cd 4          986813
LOCATION               0
Cross Street      834910
LAT                    0
LON                    0
dtype: int64


In [4]:
# Filter rows where both LAT and LON are 0
zero_values = Org_data[(Org_data['LAT'] == 0) & (Org_data['LON'] == 0)]

# Display the rows
# print(zero_values)
zero_count = zero_values.shape[0]
print(f"Number of rows with LAT and LON as 0,0: {zero_count}")
df = Org_data[~((Org_data['LAT'] == 0) & (Org_data['LON'] == 0))]



Number of rows with LAT and LON as 0,0: 2103


Standardise Column Data

In [5]:
df['Date Rptd'] = pd.to_datetime(df['Date Rptd'])   
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'])   
df['DATE OCC'][1]

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Timestamp('2020-02-08 00:00:00')

Ensure numerical fields 

In [6]:
df['Vict Age'] = pd.to_numeric(df['Vict Age'], errors='coerce')

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Remove Unnecessary Columns

In [7]:
df = df.drop(['DR_NO', 'Cross Street'], axis=1)

Add Derived Features

In [8]:
# Extract year and month from 'DATE OCC'
df['Year'] = df['DATE OCC'].dt.year
df['Month'] = df['DATE OCC'].dt.month

Encode Categorical Variables

In [9]:
# df['Vict Sex']

0           M
1           M
2           X
3           M
4         NaN
         ... 
986868    NaN
986869      X
986870      M
986871      F
986872    NaN
Name: Vict Sex, Length: 984770, dtype: object

In [10]:
df['Vict Sex'] = df['Vict Sex'].map({'M': 0, 'F': 1, 'X': 2}).fillna(3) # 'X' for unknown
df['Vict Sex']

Check for Duplicates

In [12]:
print(df.duplicated().sum())
df = df.drop_duplicates()


55977


#### Combine DATE OCC and TIME OCC into a Single Datetime Column
You must first convert DATE OCC and TIME OCC into a single Datetime column to align with Meteostat's time requirements.

In [16]:
# Function to convert TIME OCC (e.g., 630, 2130) to HH:MM format
def convert_time(time):
    time = f"{int(time):04}"  # Ensures time is zero-padded (e.g., 630 -> 0630)
    return f"{time[:2]}:{time[2:]}"  # Returns in 'HH:MM' format

# Combine DATE OCC and TIME OCC into a single datetime column
df['TIME OCC'] = df['TIME OCC'].apply(convert_time)
df['Date/Time'] = pd.to_datetime(df['DATE OCC'].astype(str) + ' ' + df['TIME OCC'], format='%Y-%m-%d %H:%M')

# Display the result
# print(df[['DATE OCC', 'TIME OCC', 'Date/Time']].head())



    DATE OCC TIME OCC           Date/Time
0 2020-03-01    21:30 2020-03-01 21:30:00
1 2020-02-08    18:00 2020-02-08 18:00:00
2 2020-11-04    17:00 2020-11-04 17:00:00
3 2020-03-10    20:37 2020-03-10 20:37:00
4 2020-09-09    06:30 2020-09-09 06:30:00


Statistical Summary

In [ ]:
# print(df.describe())


                           Date Rptd                       DATE OCC  \
count                         928793                         928793   
mean   2022-06-06 11:52:42.740891136  2022-05-27 01:03:20.028639488   
min              2020-01-01 00:00:00            2020-01-01 00:00:00   
25%              2021-04-13 00:00:00            2021-03-28 00:00:00   
50%              2022-07-19 00:00:00            2022-07-07 00:00:00   
75%              2023-07-18 00:00:00            2023-07-08 00:00:00   
max              2024-11-24 00:00:00            2024-11-24 00:00:00   
std                              NaN                            NaN   

                AREA    Rpt Dist No       Part 1-2         Crm Cd  \
count  928793.000000  928793.000000  928793.000000  928793.000000   
mean       10.236165    1070.035142       1.399177     499.775398   
min         1.000000     101.000000       1.000000     110.000000   
25%         5.000000     555.000000       1.000000     331.000000   
50%        10.0

Summary of crimes that commited.

In [ ]:
# print(df['Crm Cd Desc'].value_counts())

In [17]:
df.sample(2)

,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,LAT,LON,Year,Month,Date/Time
691670,2023-08-02,2023-08-01,23:09,11,Northeast,1183,2,753,DISCHARGE FIREARMS/SHOTS FIRED,1100 1407,...,753.0,998.0,NaN,NaN,1800 LAKE SHORE AV,34.0850,-118.2557,2023,8,2023-08-01 23:09:00
421490,2022-11-19,2022-11-19,07:50,6,Hollywood,676,2,901,VIOLATION OF RESTRAINING ORDER,0913 2038,...,901.0,NaN,NaN,NaN,800 VINE ST,34.0853,-118.3266,2022,11,2022-11-19 07:50:00
487656,2022-09-26,2022-03-08,10:10,1,Central,192,2,354,THEFT OF IDENTITY,0377 1822,...,354.0,NaN,NaN,NaN,1400 S HOPE ST,34.0393,-118.2659,2022,3,2022-03-08 10:10:00
205001,2021-09-20,2021-09-20,18:00,12,77th Street,1253,2,890,FAILURE TO YIELD,1501,...,890.0,NaN,NaN,NaN,WESTERN AV,33.9709,-118.3090,2021,9,2021-09-20 18:00:00
524846,2022-12-01,2022-11-25,17:00,9,Van Nuys,914,1,330,BURGLARY FROM VEHICLE,0344 1300 1307 0385,...,330.0,NaN,NaN,NaN,6900 N SEPULVEDA BL,34.1957,-118.4662,2022,11,2022-11-25 17:00:00


In [19]:
print(list(df.columns))

['Date Rptd', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No', 'Part 1-2', 'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'Crm Cd 1', 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'LOCATION', 'LAT', 'LON', 'Year', 'Month', 'Date/Time']


#### Fetch Hourly Weather Data Using Meteostat
You will now fetch weather data for the entire date range in your crime dataset using the Meteostat Hourly class.

1. **Define the Location (LAT, LON) and Time Range:** Since your dataset contains LAT and LON for each crime, you can fetch weather data for Los Angeles.

2. **Fetch Weather Data:** Use the Hourly class from meteostat to download the weather data.

In [21]:

# Define the coordinates for Los Angeles (example)
location = Point(34.069912, -118.351211)  # Mean LAT, LON for Los Angeles

# Determine the start and end date from your dataset
start = df['Date/Time'].min()
end = df['Date/Time'].max()

# Fetch historical weather data
data_hourly = Hourly(location, start, end)
data_hourly = data_hourly.fetch()

# Ensure the index is a DatetimeIndex and remove timezone information
data_hourly.index = data_hourly.index.tz_localize(None)

# Resample hourly data to ensure proper merging (optional step)
data_hourly = data_hourly.resample('H').ffill().reset_index()

# Select relevant columns
weather_df = data_hourly[['time', 'temp', 'precip', 'humidity', 'windspeed', 'windgust']]
weather_df = weather_df.rename(columns={
    'time': 'Date/Time', 
    'temp': 'Temperature (°C)',
    'precip': 'Precipitation (mm)',
    'humidity': 'Humidity (%)',
    'windspeed': 'Windspeed (km/h)',
    'windgust': 'Wind Gust (km/h)'
})

# Display weather data
print(weather_df.head())


KeyboardInterrupt: 

#### Merge Weather Data with Your Crime Dataset

Now that you have the weather data (weather_df) and a unified Date/Time column in your crime dataset, you can perform a left join to add the weather data.

In [ ]:
# Merge weather data with the crime dataset
merged_df = pd.merge(df, weather_df, on='Date/Time', how='left')

# Check the final dataset
print(merged_df.head())


#### Save the finalized dataset

In [ ]:
# Save the final dataset
# merged_df.to_csv('Data_sets/crime_data_with_weather.csv', index=False)